In [ ]:
%pip install pandas seaborn matplotlib

In [ ]:
import pandas as pd

JOIN_KEYS = ["año", "capitulo", "nombre_capitulo"]

presupuesto_df = (
    pd.read_csv("../datasets/pagos_salud_chile_2018-2024.csv", delimiter=",")
    .rename(columns={"devengo": "total"})
    .groupby(["año", "capitulo", "nombre_capitulo", "subtitulo", "nombre_subtitulo"])[
        ["total"]
    ]
    .agg("sum")
    .reset_index()
)

subtitulo_columns = (
    presupuesto_df[["subtitulo", "nombre_subtitulo"]]
    .sort_values(by="subtitulo")
    .drop_duplicates()["nombre_subtitulo"]
    .to_list()
)

presupuesto_df = (
    presupuesto_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=["nombre_subtitulo"],
        values=["total"],
    )
    .fillna(0)
    .reset_index()
)

presupuesto_df = pd.concat(
    [
        presupuesto_df[["año", "capitulo", "nombre_capitulo"]],
        presupuesto_df["total"][subtitulo_columns],
    ],
    axis=1,
)
presupuesto_df.columns = [
    "año",
    "capitulo",
    "nombre_capitulo",
] + presupuesto_df.columns[3:].to_list()
presupuesto_df.columns = presupuesto_df.columns.str.lower().str.replace(" ", "_")
presupuesto_df["total_gasto"] = presupuesto_df.iloc[:, 3:].sum(axis=1)

presupuesto_df

In [ ]:
YEARS = range(2018, 2024)

intento_ideacion_df = pd.read_csv("../datasets/ideacion_intento.csv", delimiter=",")
urgencia_autolesiones_df = pd.read_csv(
    "../datasets/urgencia_lesiones_autoinflingidas.csv", delimiter=","
)
casen_df = pd.read_csv("../datasets/casen.csv", delimiter=",")
egresos_df = pd.read_csv("../datasets/egresos_hospitalarios.csv", delimiter=",")

joined_df = pd.merge(presupuesto_df, intento_ideacion_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, urgencia_autolesiones_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, casen_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, egresos_df, on=JOIN_KEYS, how="outer")
joined_df = joined_df.query("año >= 2018 and año <= 2023").reset_index(drop=True)
joined_df

In [ ]:
joined_df.info()

In [ ]:
# joined_df.filter(like="hospitalizaciones").columns.to_list()

In [ ]:
# joined_df.filter(like="hospitalizacion_total").columns.to_list()

In [ ]:
joined_df[
    ["año", "capitulo", "nombre_capitulo"]
    + joined_df.filter(like="promedio").columns.to_list()
]

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_ideacion"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_intento"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["urgencias_lesiones_autoinflingidas"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_intento"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
joined_df.query("año == 2023")["urgencias_lesiones_autoinflingidas"].plot.hist(
    title="Urgencias por Lesiones Autoinflingidas - Año 2023", figsize=(6, 6)
)

In [ ]:
joined_df.query("año == 2018")["nombre_capitulo"].value_counts()

In [ ]:
# import numpy as np

# np.array_split(joined_df.filter(like="promedio").columns.to_list(), 5)[0].tolist()

In [ ]:
def get_top_k_columns_with_less_zeroes(k, pattern, df):
    promedio_variables = df.filter(like=pattern).columns.to_list()

    zero_counts = df[promedio_variables].apply(lambda col: (col == 0).sum())
    top_k = zero_counts.sort_values(ascending=True).index[:k]
    return top_k.to_list()


get_top_k_columns_with_less_zeroes(10, "promedio", joined_df)

In [ ]:
get_top_k_columns_with_less_zeroes(10, "promedio", joined_df)

In [ ]:
joined_df[
    "top_5_dias_hospitalizacion_promedio_esquizofrenia_trastornos_esquizotípicos_y_trastornos_delirantes"
].hist()

In [ ]:
import seaborn as sns

top_k_columns_with_less_zeroes = get_top_k_columns_with_less_zeroes(
    10, "hospitalizaciones", joined_df
)

ANALYSIS_VARIABLES = [
    "gastos_en_personal",
    "bienes_y_servicios_de_consumo",
    "prestaciones_de_seguridad_social",
    "transferencias_corrientes",
    "otros_gastos_corrientes",
    "adquisicion_de_activos_no_financieros",
    "iniciativas_de_inversion",
    "total_gasto",
    "suicidio_ideacion",
    "suicidio_intento",
    "urgencias_lesiones_autoinflingidas",
    "total_fonasa",
    "problemas_llegar_consulta_si",
    "problemas_conseguir_cita_si",
    "problemas_ser_atendido_si",
    "problemas_pagar_por_costo_si",
    *top_k_columns_with_less_zeroes,
]

for year in YEARS:
    plot = sns.pairplot(
        joined_df[joined_df["año"] == year],
        x_vars=ANALYSIS_VARIABLES,
        y_vars=ANALYSIS_VARIABLES,
    )

    for ax in plot.axes.flatten():
        ax.set_xlabel(ax.get_xlabel(), rotation=45)
        ax.set_ylabel(ax.get_ylabel(), rotation=45)
        ax.yaxis.get_label().set_horizontalalignment("right")

    plot.figure.suptitle(f"AÑO {year}", y=1.01, fontsize=16)

In [ ]:
import seaborn as sns

top_k_columns_with_less_zeroes = get_top_k_columns_with_less_zeroes(
    10, "promedio", joined_df
)

ANALYSIS_VARIABLES = [
    "gastos_en_personal",
    "bienes_y_servicios_de_consumo",
    "prestaciones_de_seguridad_social",
    "transferencias_corrientes",
    "otros_gastos_corrientes",
    "adquisicion_de_activos_no_financieros",
    "iniciativas_de_inversion",
    "total_gasto",
    "suicidio_ideacion",
    "suicidio_intento",
    "urgencias_lesiones_autoinflingidas",
    "total_fonasa",
    "problemas_llegar_consulta_si",
    "problemas_conseguir_cita_si",
    "problemas_ser_atendido_si",
    "problemas_pagar_por_costo_si",
    *top_k_columns_with_less_zeroes,
]

for year in YEARS:
    plot = sns.pairplot(
        joined_df[joined_df["año"] == year],
        x_vars=ANALYSIS_VARIABLES,
        y_vars=ANALYSIS_VARIABLES,
    )

    for ax in plot.axes.flatten():
        ax.set_xlabel(ax.get_xlabel(), rotation=45)
        ax.set_ylabel(ax.get_ylabel(), rotation=45)
        ax.yaxis.get_label().set_horizontalalignment("right")

    plot.figure.suptitle(f"AÑO {year}", y=1.01, fontsize=16)

In [ ]:
top_k_columns_with_less_zeroes = get_top_k_columns_with_less_zeroes(
    10, "hospitalizacion_total", joined_df
)

ANALYSIS_VARIABLES = [
    "gastos_en_personal",
    "bienes_y_servicios_de_consumo",
    "prestaciones_de_seguridad_social",
    "transferencias_corrientes",
    "otros_gastos_corrientes",
    "adquisicion_de_activos_no_financieros",
    "iniciativas_de_inversion",
    "total_gasto",
    "suicidio_ideacion",
    "suicidio_intento",
    "urgencias_lesiones_autoinflingidas",
    "total_fonasa",
    "problemas_llegar_consulta_si",
    "problemas_conseguir_cita_si",
    "problemas_ser_atendido_si",
    "problemas_pagar_por_costo_si",
    *top_k_columns_with_less_zeroes,
]

for year in YEARS:
    plot = sns.pairplot(
        joined_df[joined_df["año"] == year],
        x_vars=ANALYSIS_VARIABLES,
        y_vars=ANALYSIS_VARIABLES,
    )

    for ax in plot.axes.flatten():
        ax.set_xlabel(ax.get_xlabel(), rotation=45)
        ax.set_ylabel(ax.get_ylabel(), rotation=45)
        ax.yaxis.get_label().set_horizontalalignment("right")

    plot.figure.suptitle(f"AÑO {year}", y=1.01, fontsize=16)